In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Importing necessary libraries***

In [18]:
import pandas as pd
# import libraries and packages
import numpy as np
import pandas as pd
import re
import time
from nltk import word_tokenize
import string
from tqdm import tqdm
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from textblob import TextBlob
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GRU, Dense, Dropout


**Data loading and preprocessing**

In [5]:
data_train= pd.read_csv("/content/drive/MyDrive/PS_train.csv")
data_test= pd.read_csv("/content/drive/MyDrive/PS_dev.csv")

def clean_text(text):
    stop_words=open(r'/content/drive/MyDrive/icon_task_2020/TamilStopWords.txt', encoding='utf=8').read().split('\n')
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))
    text = "".join([word for word in text if word not in string.punctuation])
    text = "".join([word for word in text if not word.isdigit()])
    tokens = re.split('\s+',text)
    text = " ".join(word for word in tokens if word not in stop_words)          # here english words may lemmatize so kept lematization
    return text

In [6]:
print(len(data_train), len(data_test))
data_train.head()

4352 544


,content,labels
0,தென்காசி தொகுதி புதிய தமிழகம் கட்சி வேட்பாளர் ...,Neutral
1,அண்ணன் இதனை சூசகமாக 11 மாதங்கள் முன்பே பேட்டிய...,Substantiated
2,ஒரு வருடம் ஆகி விட்டது இந்த துயரம் நேர்ந்து......,Opinionated
3,"எடப்பாடியை கண்டுகொள்ளாத ""எடப்பாடி""🫢\n ---\nஆதர...",Positive
4,எங்களின் அரசியல் அடுத்த தலைமுறைக்குமானது \n#மக...,Opinionated


In [7]:
# data_train['labels'].value_counts()

In [8]:
# data_test['labels'].value_counts()

In [9]:
data_train['clean'] = data_train['content'].apply(lambda x:clean_text(x))
data_test['clean'] = data_test['content'].apply(lambda x:clean_text(x))


In [11]:
# !pip install tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocabulary_size = 30000  #
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train1 = pad_sequences(sequences, maxlen=100)
sequences = tokenizer.texts_to_sequences(X_test)
X_test1 = pad_sequences(sequences, maxlen=100)

In [28]:
# this is for label handling, encoding the label
#y_train = y_train['Label']  # Remove this line
#y_test = y_test['Label']   # Remove this line
y = pd.get_dummies(y_train).values

y_train1 = pd.get_dummies(y_train).values
y_test1 = pd.get_dummies(y_test).values
diz_label1 = {}
for i,label in enumerate(data_train.labels.factorize()[1]):
    diz_label1[i] = label

diz_label1

{0: 'Neutral',
 1: 'Substantiated',
 2: 'Opinionated',
 3: 'Positive',
 4: 'Sarcastic',
 5: 'Negative',
 6: 'None of the above'}

In [16]:
# y_train_encoded = label_encoder.fit_transform(y_train)
# y_test_encoded = label_encoder.transform(y_test)

In [12]:
embeddings_index = {}
with open("/content/drive/MyDrive/icon_task_2020/wiki.ta.vec",encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

<ipython-input-12-1756e0f1f4bb>:5: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  coefs = np.fromstring(coefs, "f", sep=" ")


Found 245374 word vectors.


In [13]:
VOCAB_SIZE = len(tokenizer.word_index) + 2
EMBEDDING_DIM = 300
hits = 0
misses =0

# Prepare embedding matrix
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        misses += 1

print("Converted %d words (%d misses)" % (hits, misses))

Converted 9082 words (8529 misses)


In [30]:
import pandas as pd
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

# Create an EarlyStopping object
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# ... (your existing code for defining model1) ...

# Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model with early stopping




**Hybrid Model**: Hybrid models in machine learning combine different types of models to leverage their strengths and improve overall performance.

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dropout, Flatten, Dense

model2 = Sequential()
model2.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True))
model2.add(LSTM(128, return_sequences=True))  #LSTM layer with return_sequences=True
model2.add(GRU(64))  #GRU layer after LSTM
model2.add(Dropout(0.2))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(7, activation='softmax'))  #Assuming 5 output classes

In [45]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(X_train1, y_train1, epochs=100, batch_size=64, validation_data=(X_test1, y_test1),callbacks=[early_stopping] )
pred2 = model2.predict(X_test1)
print(classification_report(np.argmax(y_test1,axis=1),np.argmax(pred2,axis=1)))

Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.2780 - loss: 1.8365 - val_accuracy: 0.2941 - val_loss: 1.7418
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3567 - loss: 1.6769 - val_accuracy: 0.3180 - val_loss: 1.6851
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.4464 - loss: 1.4609 - val_accuracy: 0.2923 - val_loss: 1.8269
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.5837 - loss: 1.0861 - val_accuracy: 0.2886 - val_loss: 1.8992
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6655 - loss: 0.8676 - val_accuracy: 0.2702 - val_loss: 2.1837
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.19      0.08      0.12        84
           2       0.33      0.95      0.49        20
           3       0.37      0.52      0.43       153
           4       0.21      0.19      0.20       

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**CNN+GRU Hybrid**

In [26]:
model3 = Sequential()
model3.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], input_length=100, trainable=False))
model3.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(GRU(128))
model3.add(Dropout(0.2))  # Add dropout for regularization
model3.add(Dense(7, activation='softmax'))  # Assuming binary classification



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [32]:
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.fit(
    X_train1,
    y_train1,
    epochs=10,
    batch_size=64,
    validation_data=(X_test1, y_test1),
    callbacks=[early_stopping]
)

Epoch 1/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.2975 - loss: 1.8083 - val_accuracy: 0.2941 - val_loss: 1.7177
Epoch 2/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3708 - loss: 1.6223 - val_accuracy: 0.3051 - val_loss: 1.6915
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4414 - loss: 1.4616 - val_accuracy: 0.3015 - val_loss: 1.7185
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5291 - loss: 1.2749 - val_accuracy: 0.2739 - val_loss: 1.8386
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6528 - loss: 1.0135 - val_accuracy: 0.2886 - val_loss: 1.9940


In [33]:
y_pred = model.predict(X_test1)


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [36]:
y_pred = model3.predict(X_test1)  # Predict using model3, not 'model'
y_pred_classes = np.argmax(y_pred, axis=1)  # Get predicted class labels

# Now use y_pred_classes in the classification_report
print(classification_report(y_test1.argmax(axis=1), y_pred_classes))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.29      0.06      0.10        84
           2       0.40      0.85      0.54        20
           3       0.30      0.82      0.44       153
           4       0.31      0.07      0.12        69
           5       0.24      0.10      0.14       115
           6       1.00      0.04      0.07        52

    accuracy                           0.31       544
   macro avg       0.36      0.28      0.20       544
weighted avg       0.33      0.31      0.21       544



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**LSTM**

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Flatten, Dense
model1 = Sequential()
model1.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True))
model1.add(LSTM(128, return_sequences=True))  # LSTM layer with return_sequences=True
model1.add(Dropout(0.2))
model1.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(7, activation='softmax'))

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



model1.fit(X_train1, y_train1, epochs=10, batch_size=64, validation_data=(X_test1, y_test1,),callbacks=[early_stopping] )





Epoch 1/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.2852 - loss: 1.8200 - val_accuracy: 0.2831 - val_loss: 1.7434
Epoch 2/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.3948 - loss: 1.5903 - val_accuracy: 0.2886 - val_loss: 1.7370
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5283 - loss: 1.2761 - val_accuracy: 0.3107 - val_loss: 1.8875
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6964 - loss: 0.8619 - val_accuracy: 0.2960 - val_loss: 2.1572
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8271 - loss: 0.5355 - val_accuracy: 0.2960 - val_loss: 2.4064
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  


In [43]:
pred1 = model1.predict(X_test1)
y_pred_classes = np.argmax(pred1, axis=1)  # Get predicted class labels

print(classification_report(np.argmax(y_test1,axis=1),y_pred_classes))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
              precision    recall  f1-score   support

           0       0.14      0.18      0.15        51
           1       0.18      0.27      0.22        84
           2       0.39      0.80      0.52        20
           3       0.42      0.38      0.40       153
           4       0.18      0.19      0.19        69
           5       0.37      0.33      0.35       115
           6       0.00      0.00      0.00        52

    accuracy                           0.29       544
   macro avg       0.24      0.31      0.26       544
weighted avg       0.28      0.29      0.28       544



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Meta models:Combine Predictions from Base Models**

In [63]:
from sklearn.tree import DecisionTreeClassifier  # Import DecisionTreeClassifier

stacked_predictions = np.column_stack((pred1, pred2,y_pred))#,final_predictions_rm)) # final_predictions))

meta_model2 = DecisionTreeClassifier(random_state=42)  # Create Decision Tree instance
meta_model2.fit(stacked_predictions, np.argmax(y_test1, axis=1))

final_predictions_dt = meta_model2.predict(stacked_predictions)
print(classification_report(np.argmax(y_test1, axis=1), final_predictions_dt))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        51
           1       0.97      0.99      0.98        84
           2       1.00      1.00      1.00        20
           3       1.00      0.99      0.99       153
           4       0.99      1.00      0.99        69
           5       1.00      0.98      0.99       115
           6       1.00      0.98      0.99        52

    accuracy                           0.99       544
   macro avg       0.99      0.99      0.99       544
weighted avg       0.99      0.99      0.99       544



In [83]:

from sklearn.ensemble import RandomForestClassifier
meta_model1 = RandomForestClassifier(n_estimators=1000, random_state=45)
stacked_predictions = np.column_stack((pred1, pred2,y_pred))#,final_predictions_rm)) # final_predictions))

meta_model1.fit(stacked_predictions, np.argmax(y_test1, axis=1))

# Make final predictions
final_predictions_rm = meta_model1.predict(stacked_predictions)
print(classification_report(np.argmax(y_test1, axis=1), final_predictions_rm))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        51
           1       0.98      0.98      0.98        84
           2       1.00      1.00      1.00        20
           3       0.98      1.00      0.99       153
           4       0.99      1.00      0.99        69
           5       1.00      0.98      0.99       115
           6       1.00      0.98      0.99        52

    accuracy                           0.99       544
   macro avg       0.99      0.99      0.99       544
weighted avg       0.99      0.99      0.99       544



In [85]:
from sklearn.linear_model import LogisticRegression
meta_model3 = LogisticRegression()  # Adjust parameters as needed
stacked_predictions = np.column_stack((pred1, pred2,y_pred))#,final_predictions_rm)) # final_predictions))

meta_model3.fit(stacked_predictions, np.argmax(y_test1, axis=1))  # y_train1: true labels


# Make final predictions
final_predictions_lr = meta_model3.predict(stacked_predictions)
print(classification_report(np.argmax(y_test1, axis=1), final_predictions_lr))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.18      0.06      0.09        84
           2       0.68      0.75      0.71        20
           3       0.31      0.78      0.44       153
           4       0.14      0.03      0.05        69
           5       0.40      0.34      0.37       115
           6       0.00      0.00      0.00        52

    accuracy                           0.33       544
   macro avg       0.25      0.28      0.24       544
weighted avg       0.24      0.33      0.25       544



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Load the Original Test Data**

In [54]:
test = pd.read_csv("/content/drive/MyDrive/PS_test_without_lables.csv")
test.head()
# X_train= data_train['clean']
# X_test= data_test['clean']
# y_train= data_train['labels']
# y_test= data_test['labels']

,Id,content
0,PS_01,இஸ்லாமிய சகோதரர்களுடன் ரமலான் கொண்டாடிய அதிமுக...
1,PS_02,\nஓபிஎஸ் - எடப்பாடி போட்டா போட்டி! திடீரென பணி...
2,PS_03,இன்றைய பரப்புரை:\n\nநாம் தமிழர் கட்சி தலைமை ஒர...
3,PS_04,"🇰🇬🎙️ இன்னும் 05 ஏ நாளில், வெல்வோம் தமிழராய் - ..."
4,PS_05,டாஸ்மாக்ல மட்டும்தான் கருணாநிதி அவர்களின் பெய...


In [55]:
test.head()



,Id,content
0,PS_01,இஸ்லாமிய சகோதரர்களுடன் ரமலான் கொண்டாடிய அதிமுக...
1,PS_02,\nஓபிஎஸ் - எடப்பாடி போட்டா போட்டி! திடீரென பணி...
2,PS_03,இன்றைய பரப்புரை:\n\nநாம் தமிழர் கட்சி தலைமை ஒர...
3,PS_04,"🇰🇬🎙️ இன்னும் 05 ஏ நாளில், வெல்வோம் தமிழராய் - ..."
4,PS_05,டாஸ்மாக்ல மட்டும்தான் கருணாநிதி அவர்களின் பெய...


In [56]:
test['clean'] = test['content'].apply(lambda x:clean_text(x))
test = pad_sequences(sequences, maxlen=100)

**Obtain Predictions from hybrid model**

In [58]:
pred_test1=model1.predict(test)
pred_test2=model2.predict(test)
pred_test3=model3.predict(test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


**Obtain Predictions From Meta Model**

In [87]:
stacked_predictions = np.column_stack((pred_test1, pred_test2, pred_test3))
meta_pred1 = meta_model1.predict(stacked_predictions)
meta_model2=meta_model1.predict(stacked_predictions)
meta_pred3 = meta_model3.predict(stacked_predictions)


**Decode Predictions**

In [88]:
|decoded_labels_test = [diz_label1[index] for index in y_pred_classes]

decoded_labels = [diz_label1[index] for index in meta_pred1]

In [98]:
# decoded_labels

**Create the Submission DataFrame**

In [94]:
# Load the original test DataFrame
original_test_df = pd.read_csv("/content/drive/MyDrive/PS_test_without_lables.csv")

# Create the submission DataFrame using the 'Id' column from the original test DataFrame
submission_df = pd.DataFrame({'id': original_test_df['Id'], 'Labels': decoded_labels})
submission_df.to_csv('YenCS_Tamil_run3.csv', index=False)

In [97]:
len(submission_df),len(original_test_df)

(544, 544)